In [23]:
import mlutils
import warnings
import os
import pandas as pd
warnings.filterwarnings(action="ignore")
os.getcwd()

station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
near_bus = pd.read_csv(
    "assets/near_bus_500m.csv", encoding="CP949", index_col=0
)
seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)
sub_info = pd.read_csv(
    "assets/sub_and_bike_info.csv", encoding="CP949", index_col=0
)

search_info = pd.read_csv(
    "assets/search_info.csv", encoding="CP949", index_col=0
)
moon_light = mlutils.moon_light(station,near_bus,seoul_bike,sub_info)


In [26]:
mlutils.search_by_text(search_info,'월촌중')

,value,label
529,529,목동1단지아파트 상가 앞 (월촌중학교 버스정류소 옆)


In [ ]:
bike= search_info[search_info['value'] <3000]

for i in range(100) :
    dep = search_info.sample(1)
    dep_id = dep['value'].iloc[0]
    arr = bike.sample(1)
    arr_id = arr.index[0] 
    try : 
        moon_light.route_data(dep_id,arr_id)
    except :
        print(dep, arr)
        break


In [27]:
moon_light.route_data(4443,529)





버스


IndexError: single positional indexer is out-of-bounds

In [40]:

# id를 대여소 번호로 바꿔줌
def arrival_info( arr_id: int) -> pd.DataFrame:
    return station.iloc[[arr_id]]

a = mlutils.search_by_text(search_info,'염창역')
# moon_light.departure_info(4508)

a

,value,label
486,486,KB국민은행 염창역 지점 앞
545,545,"염창역 3, 4번 출구 사이"
838,838,염창역 1번 출구
122,3122,9호선 염창역
439,4439,N26 염창역.서울도시가스
527,4527,N26 염창역.서울도시가스
1282,5282,N62 염창역.서울도시가스
1391,5391,N62 염창역.서울도시가스


> ### 디버그 용도

In [43]:
import mlutils
import warnings
import os
import pandas as pd

warnings.filterwarnings(action="ignore")
os.getcwd()

station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
near_bus = pd.read_csv("assets/near_bus_500m.csv", encoding="CP949", index_col=0)
seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)
sub_info = pd.read_csv("assets/sub_and_bike_info.csv", encoding="CP949", index_col=0)

search_info = pd.read_csv("assets/search_info.csv", encoding="CP949", index_col=0)
moon_light = mlutils.moon_light(station, near_bus, seoul_bike, sub_info)

#  def route_data(dep_id, arr_id):

dep_id = 4443
arr_id = 528
dep_info = moon_light.departure_info(dep_id)  # dataframe
arr_info = moon_light.arrival_info(arr_id)  # dataframe


if dep_id < 3000:
    print("자전거")
    bike = moon_light.route_coor(dep_info, arr_info)
    record_info = moon_light.record_info(dep_info.index[0], arr_info.index[0], 50)
    # [{"bike": ["당산 육갑문", "목동1단지아파트 118동 앞"]}]
    route_info = [
        {
            "bike": [
                dep_info["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ]
        }
    ]

    retu = dict(bike=bike, route_info=route_info)

elif dep_id > 4000:
    print("버스")

    # * -- 출도착 버스 정보-- *
    dep_arr_bus_info = moon_light.finding_start_end(dep_info, arr_info)
    print(dep_arr_bus_info)


    # * -- 경로 선택 및 경로 노선 추출 -- *
    bus_whole, waypoint, bus_start_end = moon_light.bus_route_info(dep_arr_bus_info)

    # * -- 좌표 받기 -- *
    arr_trans = bus_start_end.iloc[[1]]

    arr_bike_info = moon_light.near_bus_bike_info(arr_trans, arr_info).sort_values(
        by="num", ascending=False
    )

    bus = moon_light.bus_route_coor(bus_start_end, waypoint)
    walk = moon_light.route_coor(arr_trans, arr_bike_info.iloc[[0]])
    bike = moon_light.route_coor(arr_bike_info.iloc[[0]], arr_info)
    record_info = moon_light.record_info(
        arr_bike_info["st_id"].iloc[0], arr_info.index[0], 50
    )

    # # -- 자전거 길 네이버 지도로도 표현 -- #
    # a = arr_bike_info.iloc[[0]][["latitude", "longtitude"]]
    # b = arr_info.reset_index()[["latitude", "longtitude"]]
    # start_end = pd.concat([a, b], axis=0)
    # bus # bus_direct
    route_info = [
        {
            "bus": [
                bus_start_end["name"].iloc[0],
                bus_start_end["name"].iloc[1],
            ],
            "bike": [
                arr_bike_info["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ],
        },
    ]
    dep_bike_info = moon_light.near_bus_bike_info(
        bus_start_end.iloc[[0]], arr_info
    ).sort_values(by="num", ascending=False)
    if dep_bike_info.empty == False:
        bike2 = moon_light.route_coor(dep_bike_info, arr_info)
        record_info2 = moon_light.record_info(
            dep_bike_info["st_id"].iloc[0], arr_info.index[0], 50
        )
        bike2 = (
            {
                "bike2": [
                    dep_bike_info["st_name"].iloc[0],
                    arr_info["st_name"].iloc[0],
                    record_info2[0],
                    record_info2[1],
                ]
            },
        )
        route_info.extend(bike2)

    retu = dict(
        bus=bus,
        walk=walk,
        bike=bike,
        bike2=bike2,
        route_info=route_info,
    )

else:
    print("지하철")
    # * -- 자전거 대여소 정보 추출 -- *
    near_bus_bike_info = moon_light.near_bus_bike_info(dep_info, arr_info).sort_values(
        by="dist"
    )
    if near_bus_bike_info.empty :
        print('near_bus_bike_info is empty')
    departure_bike_station = pd.DataFrame(near_bus_bike_info.iloc[0]).T

    # * -- 자전거 경로(사실 보행로 추천이라는 사실~)-- *
    sub = moon_light.route_coor(dep_info, departure_bike_station)
    bike = moon_light.route_coor(departure_bike_station, arr_info)
    record_info = moon_light.record_info(
        departure_bike_station["st_id"].iloc[0], arr_info.index[0], 50
    )
    # sub
    # [{"sub": ["당산역"], "bike": ["당산 육갑문", "목동1단지아파트 118동 앞"]}]
    route_info = [
        {
            "sub": [dep_info["sub_name"].iloc[0]],
            "bike": [
                departure_bike_station["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ],
        }
    ]
    retu = dict(sub=sub, bike=bike, route_info=route_info)


버스
                           bike_id  bus order     id        name  longtitude  \
524              [247,267,268,272]  N26   109  19176         당산역   126.90241   
527  [700,773,1169,1171,2703,3761]  N26   112  16001  염창역.서울도시가스  126.875881   

      latitude        dist  
524  37.534863         NaN  
527  37.547097  665.465568  


In [57]:
from typing import List




center_data(retu['bus'])

array([ 37.5384807, 126.8996431])

In [ ]:
dep_info = moon_light.departure_info(dep_id)  # dataframe
arr_info = moon_light.arrival_info(arr_id)  # dataframe
# moon_light.finding_start_end(dep_info, arr_info)
dep_info

In [ ]:
#  def route_data(dep_id, arr_id):

dep_id = 4508
arr_id = 528
dep_info = moon_light.departure_info(dep_id)  # dataframe
arr_info = moon_light.arrival_info(arr_id)  # dataframe

if dep_id < 3000:
    print("자전거")
    bike_coor = moon_light.route_coor(dep_info, arr_info)
    print(dep_info)
    retu = dict(bike_coor=bike_coor)

elif dep_id > 4000:
    print("버스")
    # * -- 자전거 대여소 정보 추출 -- *

    # * -- 출도착 버스 정보-- *
    dep_arr_bus_info = moon_light.finding_start_end(dep_info, arr_info)


In [ ]:

    # * -- 경로 선택 및 경로 노선 추출 -- *
    bus_whole, waypoint, bus_start_end = moon_light.bus_route_info(dep_arr_bus_info)

    # * -- 좌표 받기 -- *


    arr_trans = bus_start_end.iloc[[1]]

    dep_bike_info = moon_light.near_bus_bike_info(
    bus_start_end.iloc[[0]], arr_info
    ).sort_values(by="num", ascending=False)

    arr_bike_info = moon_light.near_bus_bike_info(
    arr_trans, arr_info
    ).sort_values(by="num", ascending=False)

    dep_arr_bike_coor = moon_light.route_coor(dep_bike_info, arr_info)


    bus_coor = moon_light.bus_route_coor(bus_start_end, waypoint)
    walk_coor = moon_light.route_coor(
        arr_trans, arr_bike_info.iloc[[0]]
    )
    bike_coor = moon_light.route_coor(arr_bike_info.iloc[[0]], arr_info)
    retu= dict(
        dep_arr_bike_coor=dep_arr_bike_coor,
        bus_coor=bus_coor,
        walk_coor=walk_coor,
        bike_coor=bike_coor,
    )
else:
    print("지하철")
    # * -- 자전거 대여소 정보 추출 -- *
    near_bus_bike_info = moon_light.near_bus_bike_info(
        dep_info, arr_info
    ).sort_values(by="dist")
    departure_bike_station = pd.DataFrame(near_bus_bike_info.iloc[0]).T

    # * -- 자전거 경로(사실 보행로 추천이라는 사실~)-- *
    walk_coor = moon_light.route_coor(dep_info, departure_bike_station)
    bike_coor = moon_light.route_coor(departure_bike_station, arr_info)
    retu= dict(
        walk_coor=walk_coor, bike_coor=bike_coor
    )

> ### Card에 넣을 내용

In [4]:

from typing import Dict

def record_check(dep_id: int,arr_id: int,count : int = 50) -> pd.DataFrame :
    BM = (seoul_bike['st_id1'] == dep_id) & (seoul_bike['st_id2'] == arr_id)
    raw_data_1 = seoul_bike[BM]
    BM = (seoul_bike['st_id1'] == arr_id) & (seoul_bike['st_id2'] == dep_id)
    raw_data_2 = seoul_bike[BM]
    concat_data = pd.concat([raw_data_1,raw_data_2],axis=0)

    if len(concat_data) <= count :
        return pd.DataFrame([])

    
    all_time = concat_data['riding_time'].value_counts().sort_values(ascending=False) # 도달한 시간대
    total_borrow =len(concat_data)

    k = []
    i = 2
    # 일정 비율 이상인 시간대만 구하기
    while len(k) < 1:
        k = all_time[all_time >= (total_borrow / i)]
        i = i * 1.5
        
    ### 대여시간대
    time_line = k.index

    ### 대여기록
    counts = k.values

    total_counts = k.sum()

    ### 대여시간 * 대여기록
    total_time = sum([a * b for a, b in zip(time_line, counts)])

    # 평균 시간
    avg_time = total_time / total_counts

    # 올림
    result = round(avg_time, 0)

    return [result,total_borrow]
dep_info = moon_light.departure_info(4443)
arr_info = moon_light.arrival_info(528)


record_check(arr_bike_info['st_id'].iloc[0],arr_info.index[0])

NameError: name 'arr_bike_info' is not defined

In [28]:
# bus # bus_direct
[{"bus":[bus_start_end['name'].iloc[0],bus_start_end['name'].iloc[1]],'bike':[arr_bike_info['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}, {'bike2' : [dep_bike_info['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}]

# sub
[{"sub":['당산역'],'bike':['당산 육갑문', '목동1단지아파트 118동 앞']}]
[{"sub":[dep_info['sub_name'].iloc[0]],'bike':[departure_bike_station['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}]


# bike
[{'bike':['당산 육갑문', '목동1단지아파트 118동 앞']}]
[{'bike' : [dep_info['name'].iloc[0],arr_info['st_name'].iloc[0]]}]

33.0

In [8]:
print("bus_start_end", bus_start_end['name'].iloc[0])
print('')
print("arr_trans", arr_trans.values)
print('')
print("arr_bike_info", arr_bike_info['st_name'])
print('')
print("arr_info", arr_info['st_name'].iloc[0])
print('')
print("dep_info", dep_info)
print('')
print("dep_bike_info", dep_bike_info)
print('')
print("departure_bike_station", departure_bike_station)
print('')

bus_start_end 당산역

arr_trans [['염창역.서울도시가스' 126.875881 37.547097 112 '[700,773,1169,1171,2703,3761]']]

arr_bike_info 1    염창역 3, 4번 출구 사이
2          염창역 1번 출구
0    KB국민은행 염창역 지점 앞
Name: st_name, dtype: object

arr_info 목동1단지아파트 118동 앞 (월촌초등학교 정류소 옆

dep_info                         bike_id  bus  order     id name  longtitude   latitude
443  [247,267,268,272,290,3201]  N26     24  19172  당산역  126.900719  37.534589

dep_bike_info   st_name   latitude  longtitude  st_id  time  num    dist
0   당산육갑문  37.535339  126.903679    272  33.0  109  3.03km



NameError: name 'departure_bike_station' is not defined

> ### 이름으로 버스 정류장과 대여소 선택